In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_parquet('data/processed/train_merged.parquet')
train.shape
train.isFraud.value_counts()

isFraud
0    569877
1     20663
Name: count, dtype: int64

In [3]:
# Key Entity Columns

entity_cols = [
    col for col in train.columns
    if col.startswith('card') or
       col.startswith('addr') or
       "email" in col.lower() or
       "Device" in col or
       "id_" in col
]

entity_cols

['card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'P_emaildomain',
 'R_emaildomain',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

In [4]:
train[entity_cols].head()

,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,outlook.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,yahoo.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,gmail.com,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [33]:
# Checking for the missing values. Only for the useful ones not all columns

missing = train[entity_cols + ['TransactionAmt', 'dt', 'isFraud']].isnull().mean().sort_values(ascending=False)
missing.head(20)
missing.to_csv('data/processed/missing_summary.csv', index=False)

In [6]:
# Cardinality Check
train[entity_cols].nunique().sort_values(ascending=True).head(20)

id_37          2
id_35          2
id_29          2
id_28          2
id_27          2
id_16          2
DeviceType     2
id_36          2
id_12          2
id_38          2
id_23          3
id_15          3
id_34          4
id_32          4
card4          4
card6          4
id_24         12
id_04         15
id_18         18
id_03         24
dtype: int64

In [34]:
# Cardinality Check
unique_values = train[entity_cols].nunique().sort_values(ascending=False)
unique_values.to_csv('data/processed/cardinality_summary.csv', index=False)

In [8]:
# Correlation / Mutual Information Overview

from sklearn.feature_selection import mutual_info_classif

sample = train.sample(50000)
X = sample.select_dtypes(include=[ 'int32', 'float32']).drop('isFraud', axis=1)
y = sample['isFraud']

mi = mutual_info_classif(X.fillna(0), y)
pd.Series(mi, index=X.columns).sort_values(ascending=False).head(20)

V201     0.020742
card1    0.020520
V257     0.020500
V258     0.020376
V200     0.019137
V246     0.019019
card3    0.018728
V244     0.018670
V242     0.018490
V187     0.018394
V243     0.018353
addr2    0.018292
V45      0.018285
V44      0.018041
V86      0.017970
card2    0.017726
V87      0.017591
V188     0.017393
C4       0.017330
V189     0.017321
dtype: float64

In [10]:
# Extract time columns for behavioural windows

train['hour'] = train['dt'].dt.hour
train['day'] = (train['TransactionDT'] // (24*60*60)).astype(int)
train['week'] = train['dt'].dt.isocalendar().week.astype(int)

In [31]:
train.head(8)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,dt,hour,day,week
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:00:00,0,1,48
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:00:01,0,1,48
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:01:09,0,1,48
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:01:39,0,1,48
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,2017-12-02 00:01:46,0,1,48
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:01:50,0,1,48
6,2987006,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:02:02,0,1,48
7,2987007,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-02 00:02:09,0,1,48


In [32]:
train.loc[(train['card1']==12695), ['card1', 'addr2', 'DeviceInfo', 'P_emaildomain', 'isFraud']]

,card1,addr2,DeviceInfo,P_emaildomain,isFraud
7,12695,87.0,NaN,mail.com,0
150,12695,87.0,NaN,hotmail.com,0
306,12695,87.0,NaN,gmail.com,0
670,12695,87.0,NaN,hotmail.com,0
966,12695,87.0,NaN,NaN,0
...,...,...,...,...,...
590346,12695,87.0,NaN,gmail.com,0
590371,12695,87.0,NaN,yahoo.com,0
590373,12695,87.0,NaN,yahoo.com,0
590378,12695,87.0,NaN,yahoo.com,0


In [29]:
train.addr2.unique()

array([ 87.,  nan,  96.,  35.,  60.,  98.,  43.,  65.,  32.,  13.,  31.,
       101.,  24.,  16.,  15.,  19.,  71.,  59., 102.,  44.,  26.,  69.,
        47.,  78.,  88.,  66.,  72.,  22.,  57.,  25.,  17.,  30.,  29.,
        21.,  14.,  49.,  83.,  75.,  34.,  86.,  48.,  68.,  23.,  70.,
        62.,  54.,  50.,  52.,  39.,  76.,  10.,  73.,  97.,  63.,  27.,
        28.,  38.,  74.,  77.,  92.,  79.,  84.,  82.,  40.,  36.,  46.,
        18.,  20.,  89.,  61.,  94., 100.,  55.,  51.,  93.],
      dtype=float32)

In [24]:
train.loc[(train['card1']==13926) & (train['addr1']==264), ['card1', 'addr1', 'DeviceInfo', 'P_emaildomain', 'isFraud']]

,card1,addr1,DeviceInfo,P_emaildomain,isFraud
39562,13926,264.0,Windows,hotmail.com,0
192305,13926,264.0,iOS Device,suddenlink.net,0
431723,13926,264.0,NaN,aol.com,0
433991,13926,264.0,NaN,aol.com,0
434125,13926,264.0,NaN,juno.com,0
434391,13926,264.0,NaN,aol.com,0
436700,13926,264.0,NaN,aol.com,0
436713,13926,264.0,NaN,aol.com,0
437575,13926,264.0,NaN,yahoo.com,0
437577,13926,264.0,NaN,yahoo.com,0


- PRIMARY_ENTITY = 'card1'
- SECONDARY_ENTITY = ['card1','addr1']
- DEVICE_ENTITY = 'DeviceInfo'
- EMAIL_ENTITY = 'P_emaildomain'